<a href="https://colab.research.google.com/github/ekrombouts/gcai_zuster_fietje/blob/main/notebooks/341_VraagHetZusterFietjeQ4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q datasets bitsandbytes


In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
import random

In [ ]:
# Load pre-trained fine-tuned model and tokenizer
model_id = "ekrombouts/zuster_fietje_q4"
peft_model_id = "ekrombouts/zuster_fietje_peft_q4"

# Load the base model
model = AutoModelForCausalLM.from_pretrained(model_id)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)

tokenizer.pad_token = tokenizer.eos_token
tokenizer.truncation_side = 'left'


In [ ]:
# Load dataset
path_hf_sampc = "ekrombouts/Gardenia_instruct_dataset"
dataset = load_dataset(path_hf_sampc)
# We will be working with the validation dataset
val_dataset = dataset['validation']

In [ ]:
def create_prompt(row: dict, add_response: bool = True) -> str:
    """
    Generates a prompt based on the input data in 'row'.

    Args:
        row (dict): A dictionary containing 'context', 'instruction', and optionally 'response'.
        add_response (bool): If True, the prompt will include the 'response'.
                             If False, only 'context' and 'instruction' will be included.

    Returns:
        str: The generated prompt in text format.
    """
    # Base prompt (without response)
    prompt = f"""|CONTEXT|
{row.get('context', '')}

|INSTRUCTION|
{row.get('instruction', '')}

|RESPONSE|
"""

    # Append response if 'add_response' is True and 'response' exists
    if add_response and 'response' in row:
        prompt += f"\n{row['response']}\n"

    return prompt

In [ ]:
def tokenize_prompt(model, prompt):
    # Tokenize and prepare input
    return tokenizer(prompt, return_tensors="pt").input_ids.to(model.device), \
           tokenizer(prompt, return_tensors="pt", padding=True).attention_mask.to(model.device)

def generate_output(model, input_ids, attention_mask):
    with torch.no_grad():
        output = model.generate(
            input_ids,
            attention_mask=attention_mask,
            max_new_tokens=1024,
            do_sample=True,
            top_p=1,
            top_k=50,
            temperature=0.1,
            num_return_sequences=1,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.eos_token_id
        )
    return tokenizer.decode(output[0], skip_special_tokens=True)

def answer(model, prompt):
    input_ids, attention_mask = tokenize_prompt(model, prompt)
    generated_text = generate_output(
        model=model,
        input_ids=input_ids,
        attention_mask=attention_mask)
    return generated_text[len(prompt):].strip()

In [ ]:
# Prepare the prompt with notes from sample
# row = val_dataset[1]
row = random.choice(val_dataset)  # Select a random row from the training dataset

prompt = create_prompt(
    row=row,
    add_response=False
)
print(prompt)

# Display the generated response and actual response
ref_response = row['response']  # Reference response from dataset
print("\nREFERENCE RESPONSE:")
print(ref_response)

print(f"\n{100*'-'}")
print("ZUSTER FIETJE:")
print(answer(model,prompt))
print(f"\n{100*'-'}")

In [ ]:
prompt = """|CONTEXT|

U was inc van urine. U was niet vriendelijk tijdens het verschonen.
Mw was vanmorgen incontinent van dunne def, bed was ook nat. Mw is volledig verzorgd, bed is verschoond,
Mw. haar kledingkast is opgeruimd.
Mw. zei:"oooh kind, ik heb zo'n pijn. Mijn benen. Dat gaat nooit meer weg." Mw. zat in haar rolstoel en haar gezicht trok weg van de pijn en kreeg traanogen. Mw. werkte goed mee tijdens adl. en was vriendelijk aanwezig. Pijn. Mw. kreeg haar medicatie in de ochtend, waaronder pijnstillers. 1 uur later adl. gegeven.
Ik lig hier maar voor Piet Snot. Mw. was klaarwakker tijdens eerste controle. Ze wilde iets, maar wist niet wat. Mw. een slokje water gegeven en uitgelegd hoe ze kon bellen als ze iets wilde. Mw. pakte mijn hand en bedankte me.
Mevr. in de ochtend ondersteund met wassen en aankleden. Mevr was rustig aanwezig.
Mw is volledig geholpen met ochtendzorg, mw haar haren zijn gewassen. Mw haar nagels zijn kort geknipt.
Mevr heeft het ontbijt op bed genuttigd. Daarna mocht ik na de tweede poging Mevr ondersteunen met wassen en aankleden.
Vanmorgen met mw naar buiten geweest om een sigaret te roken. Mw was niet erg spraakzaam en mw kwam op mij over alsof ze geen behoefte had aan een gesprek. Mw kreeg het koud door de wind en wilde snel weer naar binnen.

|INSTRUCTION|
Noem de 3 belangrijkste zorgproblemen

|RESPONSE|
"""

In [ ]:
print("ZUSTER FIETJE:")
print(answer(model,prompt))
print(f"\n{100*'-'}")
